# Implementation of [arXiv:1905.13729](https://arxiv.org/abs/1905.13729) 
See also: DOI: [10.1103/PhysRevD.101.095032](http://doi.org/10.1103/PhysRevD.101.095032)
## General solution to the U(1) anomaly equations
Let a vector $\boldsymbol{z}$ with $N$ non-zero integer entries such that
$$ \sum_{i=1}^N z_i=0\,,\qquad \sum_{i=1}^N z_i^3=0\,.$$
We like to build this set of $N$ integers from two subsets $\boldsymbol{\ell}$ and $\boldsymbol{k}$ with sizes

\begin{align}
\operatorname{dim}(\boldsymbol{\ell})=&
\begin{cases}
\alpha=\frac{N}{2}-1\,, & \text{ if $N$  even } \\
\beta=\frac{N-3}{2}\,, & \text{ if $N$  odd }\\
\end{cases};&
\operatorname{dim}(\boldsymbol{k})=&
\begin{cases}
\alpha=\frac{N}{2}-1\,, & \text{ if $N$  even } \\
\beta+1=\frac{N-1}{2}\,, & \text{ if $N$  odd }\\
\end{cases};
\end{align}

* $N$ even: Consider the following two vector-like examples of $\boldsymbol{z}$ such that
\begin{align}
\boldsymbol{x}=&\left(\ell_{1}, {k}_{1}, \ldots, k_{\alpha},-\ell_{1},-k_{1}, \ldots,-k_{\alpha}\right)\\
\boldsymbol{y}=&\left(0,0,\ell_{1},  \ldots, \ell_{\alpha},-\ell_1, \ldots,-\ell_{\alpha}\right)\,.
\end{align}
* $N$ odd:
$$
\begin{array}{l}
\boldsymbol{x}=\left(0, k_{1}, \ldots, k_{\beta+1},-k_{1}, \ldots,-k_{\beta+1}\right) \\
\boldsymbol{y}=\left(\ell_{1}, \ldots, \ell_{\beta}, k_{1}, 0,-\ell_{1}, \ldots,-\ell_{\beta},-k_{1}\right)
\end{array}
$$

From any of this, we can build a final $\boldsymbol{z}$ which can includes _chiral_ solutions
$$
\boldsymbol{x} \oplus \boldsymbol{y} \equiv\left(\sum_{i=1}^{N} x_{i} y_{i}^{2}\right)\boldsymbol{x}-\left(\sum_{i=1}^{N} x_{i}^{2} y_{i}\right)\boldsymbol{y}\,.
$$

## Python implmentation
Obtain a numpy array `z` of `N` integers which satisfy the Diophantine equations
```python
>>> z.sum()
0
>>> (z**3).sum()
0
```
The input is two lists `l` and `k` with any `(N-3)/2` and `(N-1)/2` integers for `N` odd, or `N/2-1` and `N/2-1` for `N` even (`N>4`).
The function is implemented below under the name: `free(l,k)`

<a href="https://colab.research.google.com/github/restrepo/anomaly/blob/main/anomalysolutions-class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://stackoverflow.com/a/43793179/2268280

In [87]:
import pandas as pd
import numpy as np
from astropy.table import Table
import itertools
import sys
import os
from functools import reduce
import warnings
warnings.filterwarnings("ignore")

In [88]:
!pip install anomalies 2>/dev/null > /dev/null

In [89]:
from anomalies import anomaly
from anomalytools import *

In [90]:
anomaly.free([-1,1],[4,-2])

array([  3,   3,   3, -12, -12,  15])

In [91]:
anomaly.free.gcd

3

In [92]:
anomaly.free.simplified

array([ 1,  1,  1, -4, -4,  5])

In [93]:
z=anomaly.free

## Analysis
`solutions` class → Initialize the object to obtain anomaly free solutions for any set of `N` integers

In [220]:
#TODO: inherit from free class
import sys
def _get_chiral(q,q_max=np.inf):
    #Normalize to positive minimum
    if q[0]<0 or (q[0]==0 and q[1]<0):
        q=-q
    #Divide by GCD
    GCD=np.gcd.reduce(q)
    q=(q/GCD).astype(int)
    if ( #not 0 in z and 
          0 not in [ sum(p) for p in itertools.permutations(q, 2) ] and #avoid vector-like and multiple 0's
          #q.size > np.unique(q).size and # check for at least a duplicated entry
          np.abs(q).max()<=q_max
           ):
        return q,GCD
    else:
        return None,None
class solutions(object):
    '''
    Obtain anomaly free solutions with N chiral fields
    
    Call the initialize object with N and get the solutions:
    Example:
    >>> s=solutions()
    >>> s(6) # N = 6
    
    Redefine the self.chiral function to implement further restrictions:
    inherit from this class and define the new chiral function
    '''
    def __init__(self,nmin=-2,nmax=2,zmax=np.inf):
        self.nmin=nmin
        self.nmax=nmax
        self.zmax=zmax
        self.CALL=False
        self.free=[]


    def __call__(self,N,*args,**kwargs):
        if kwargs.get('nmin'):
            self.nmin=kwargs['nmin']
        if kwargs.get('nmax'):
            self.nmax=kwargs['nmax']            
        if kwargs.get('zmax'):
            self.zmax=kwargs['zmax']                        
        self.CALL=True
        self.N=N
        if N%2!=0: #odd
            N_l=(N-3)//2
            N_k=(N-1)//2
        else: #even
            N_l=N//2-1
            N_k=N_l
        r=range(self.nmin,self.nmax+1)
        self.ls=list(itertools.product( *(r for i in range(N_l)) ))
        self.ks=list(itertools.product( *(r for i in range(N_k)) ))
        return self.chiral(*args,**kwargs)
        
        
    def chiral(self,*args,**kwargs):
        if not self.CALL:
            sys.exit('Call the initialized object first:\n>>> s=solutions()\n>>> self(5)')
        self.list=[]
        solt=[]
        for l in self.ls:
            for k in self.ks:
                l=list(l)
                k=list(k)
                q,gcd=_get_chiral( z(l,k) )
                #print(z(l,k))
                if q is not None and list(q) not in self.list and list(-q) not in self.list:
                    self.list.append(list(q))
                    solt.append({'l':l,'k':k,'z':list(q),'gcd':gcd})
        return solt

Chiral solutions for l and k in the range [-2,2]

In [221]:
s=solutions()

solutions for $N=5$ integers

In [222]:
pd.DataFrame(  s(5)  )

,l,k,z,gcd
0,[-2],"[-1, 2]","[2, 4, -7, -9, 10]",1
1,[-2],"[2, -1]","[1, 5, -7, -8, 9]",4


In [237]:
class dark(solutions):
    '''
    Modify the self.chiral function to obtain solutions
    with either duplicate or triplicate integers
    '''
    def chiral(self,X=False,verbose=False,print_step=100000,**kwargs):
        m=2
        if X:
            m=3
        self.list=[]
        solt=[]
        tot=len(self.ls)*len(self.ks)
        i=0
        for l in self.ls:
            for k in self.ks:
                if verbose:
                    i=i+1
                    if i%print_step==0:
                        print('{}/{} → {}'.format(i,tot,len(solt)))
                l=list(l)
                k=list(k)
                q,gcd=_get_chiral( z(l,k) )
                #print(z(l,k))
                if (q is not None and 
                    list(q) not in self.list and list(-q) not in self.list and
                    1 in [ len(set(p)) for p in itertools.permutations(q, m) ] and
                    #q.size-np.unique(q).size>m
                    np.abs(q).max()<=self.zmax
                   ):
                    self.list.append(list(q))
                    d={'n':self.N,'l':l,'k':k,'solution':list(q),'gcd':gcd}
                    solt.append(d)
                    self.free.append(d)
        return solt    

Chiral solutions with repeated integers

Example: Force solutions with triplicate integers

In [240]:
%%time
slt=s(6,X=True,nmin=-10,nmax=10,zmax=32)
slt=

CPU times: user 18.3 s, sys: 0 ns, total: 18.3 s
Wall time: 18.3 s


[{'n': 6,
  'l': [-10, 5],
  'k': [-2, 4],
  'solution': [1, 1, 1, -4, -4, 5],
  'gcd': 1500},
 {'n': 6,
  'l': [-10, 5],
  'k': [3, 4],
  'solution': [3, 3, 3, -10, -17, 18],
  'gcd': 250}]

Full solutions

WARNING: Take two days

In [243]:
s=dark()
slt=s(6,verbose=True,print_step=500000,nmin=-21,nmax=21,zmax=32)
slt=s(7,verbose=True,print_step=1000000,nmin=-13,nmax=13,zmax=32)
slt=s(8,verbose=True,print_step=1000000,nmin=-12,nmax=12,zmax=32)
slt=s(9,verbose=True,print_step=1000000,nmin=-7,nmax=7)

500000/3418801 → 36
1000000/3418801 → 38
1500000/3418801 → 38
2000000/3418801 → 38
2500000/3418801 → 38
3000000/3418801 → 38
1000000/14348907 → 206
2000000/14348907 → 235
3000000/14348907 → 247
4000000/14348907 → 248
5000000/14348907 → 248
6000000/14348907 → 249
7000000/14348907 → 249
8000000/14348907 → 249
9000000/14348907 → 249
10000000/14348907 → 249
11000000/14348907 → 249
12000000/14348907 → 249
13000000/14348907 → 249
14000000/14348907 → 249


In [257]:
df=pd.DataFrame(s.free)

#df=df.append( pd.read_json( 'solutions8.json'  )  
#    ).append( pd.read_json( 'solutions9.json'  )   
#    ).append( pd.read_json( 'missing.json'  )
#    ).reset_index(drop=True)   

In [261]:
df.to_json('solutions.json')